In [4]:
import os
import pickle
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

In [2]:
PATH_DATA = '../data/'
PATH_MODEL = '../model/'

MODEL = 'model.pkl'
DETAILED_CLAIM = 'claim_detail.csv'
PRESCRIPTION = 'prescription_drugs.csv'
DETAILED_PRESCRIPTION = 'prescription_detail.csv'
DETAILED_PRESCRIPTION_LE = 'prescription_detail_le.csv'
DETAILED_PRESCRIPTION_LE_MAPPING = 'prescription_detail_le_mapping.pkl'

In [5]:
def prep_data():
    df_claim = pd.read_csv(os.path.join(PATH_DATA, DETAILED_CLAIM))
    df_prescription = pd.read_csv(os.path.join(PATH_DATA, PRESCRIPTION))
    df_claim = df_claim.dropna()
    df_prescription = df_prescription.dropna()

    df_prescription['date_svc'] = pd.to_datetime(df_prescription['date_svc'])
    df_claim['date_svc'] = pd.to_datetime(df_claim['date_svc'])
    df_prescription = df_prescription.sort_values(by = ['date_svc'])
    df_claim = df_claim.sort_values(by = ['date_svc'])

    merged_df = pd.merge_asof(df_prescription, df_claim, on = 'date_svc', by = 'member_id', direction = 'backward')

    merged_df = merged_df.drop(['record_id', 'member_id', 'date_svc', 'ndc', 'Unnamed: 0', 'diag', 'diag_desc', 'ccs_1_desc', 'ccs_2_desc'], axis = 1)
    merged_df = merged_df.dropna()

    merged_df.to_csv(os.path.join(PATH_DATA, DETAILED_PRESCRIPTION))

In [6]:
prep_data()

In [6]:
df = pd.read_csv(os.path.join(PATH_DATA, DETAILED_PRESCRIPTION))

In [8]:
df.head()

,Unnamed: 0,drug_category,drug_group,drug_class,ccs_3_desc
0,29084,Diuretics,Thiazides and Thiazide-Like Diuretics,Thiazides and Thiazide-Like Diuretics,Joint disorders and dislocations; trauma-related
1,40043,Diuretics,Thiazides and Thiazide-Like Diuretics,Thiazides and Thiazide-Like Diuretics,Joint disorders and dislocations; trauma-related
2,47169,Thyroid Agents,Thyroid Hormones,Thyroid Hormones,Joint disorders and dislocations; trauma-related
3,53039,Corticosteroids,Glucocorticosteroids,Glucocorticosteroids,Other non-traumatic joint disorders
4,53055,Musculoskeletal Therapy Agents,Central Muscle Relaxants,Central Muscle Relaxants,Other non-traumatic joint disorders


In [9]:
print(df.shape)
print('Unique values in each columns:')
print('drug_category', df['drug_category'].nunique())
print('drug_group', df['drug_group'].nunique())
print('drug_class', df['drug_class'].nunique())
print('ccs_3_desc', df['ccs_3_desc'].nunique())

(2258942, 5)
Unique values in each columns:
drug_category 92
drug_group 445
drug_class 653
ccs_3_desc 274


In [16]:
df = pd.get_dummies(df, columns=['drug_category', 'drug_group', 'drug_class'])

In [17]:
df.head()

,Unnamed: 0,ccs_3_desc,drug_category_Adhd/Anti-narcolepsy/Anti-obesity/Anorexiants,drug_category_Allergenic Extracts/Biologicals Misc,drug_category_Alternative Medicines,drug_category_Aminoglycosides,drug_category_Analgesics - Anti-Inflammatory,drug_category_Analgesics - NonNarcotic,drug_category_Analgesics - Opioid,drug_category_Androgens-Anabolic,...,drug_class_Vitamin B-3,drug_class_Vitamin B-6,drug_class_Vitamin C,drug_class_Vitamin D,drug_class_Vitamin K,drug_class_Wound Care - Growth Factor Agents,drug_class_Wound Care Combinations,drug_class_Wound Dressings,drug_class_Xanthines,drug_class_Zinc
0,1,Essential hypertension,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2,Asthma,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,3,Essential hypertension,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,4,Contraceptive and procreative management,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,7,Essential hypertension,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [7]:
label_encoder = LabelEncoder()
label_mapping = {}

for col in ['drug_category', 'drug_group', 'drug_class', 'ccs_3_desc']:
    df[col] = label_encoder.fit_transform(df[col])
    label_mapping[col] = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))


In [8]:
df.head()

,Unnamed: 0,drug_category,drug_group,drug_class,ccs_3_desc
0,29084,48,412,596,144
1,40043,48,412,596,144
2,47169,83,416,606,144
3,53039,42,210,310,211
4,53055,69,147,211,211


In [9]:
with open(os.path.join(PATH_DATA, DETAILED_PRESCRIPTION_LE_MAPPING), 'wb') as f:
    pickle.dump(label_mapping, f)

df.to_csv(os.path.join(PATH_DATA, DETAILED_PRESCRIPTION_LE))

In [10]:
X = df[['drug_category', 'drug_group', 'drug_class']]
y = df['ccs_3_desc']
print(X.shape, y.shape)

(2258942, 3) (2258942,)


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [12]:
clf = RandomForestClassifier()

In [13]:
clf.fit(X_train, y_train)

RandomForestClassifier()

In [16]:
with open(os.path.join(PATH_MODEL, MODEL), 'wb') as f:
    pickle.dump(clf, f)

In [14]:
y_pred = clf.predict(X_test)

In [15]:
accuracy = accuracy_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)

print('Accuracy:', accuracy)
# print('F1:', f1)
# print('Precision:', precision)
# print('Recall:', recall)

Accuracy: 0.1731936811210543


In [64]:
inp = pd.Series([12, 21, 43])
inp = inp.values.reshape(1, -1)

clf.predict(inp)
# print(X_test['drug_category'])

C:\Users\sharh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


array([218])